In [1]:
import sys
import os
IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR)

In [2]:
from imatools.common import vtktools as vtku
from imatools.common import ioutils as iou

print(iou.chooseplatform())

macOS


In [3]:

# Locations of hard drive based on platform
dirdic={'macOS'   : '/Volumes/sandisk', 
        'Linux'   : '/media/jsl19/sandisk' , 
        'Windows' : 'D:/'}

p2f = iou.fullfile(dirdic[iou.chooseplatform()], '09-dnav_vs_inav/umc/local')
N = os.listdir(p2f)
xnav = lambda x : 'LGE_' + x + 'NAV'


# Project iNAV values on dNAV mesh

In [5]:
n = 1  # 0..22 (23 cases)
case_path = iou.fullfile(p2f, str(N[n]))
inav = iou.fullfile(case_path, xnav('i'), 'OUTPUT')
dnav = iou.fullfile(case_path, xnav('d'), 'OUTPUT')

dpath = iou.searchFileByType(dnav, 'Normalised', 'vtk')[0]
ipath = iou.searchFileByType(inav, 'Normalised', 'vtk')[0]

msh_d = vtku.readVtk(dpath) # source
msh_i = vtku.readVtk(ipath) # target

msh_i_on_d = vtku.projectCellData(msh_source=msh_d, msh_target=msh_i)

vtku.writeVtk(msh_i_on_d, dnav, "Normalised_iNav_on_dNav")
        

# Compare meshes and projections 
We compare meshes

+ `msh_d = ` dNAV , and 
+ `msh_i_on_d = ` iNAV 

Both have been loaded from the `Normalised` version, so thresholds are simple, e.g `1.2, 1.3`, etc


In [ ]:
## create thresholded masks of each mesh
import vtk 

print(case_path)

d_thres = 1.2 
iod_thres = 1.15 # try different thresholds 

omsh = vtk.vtkPolyData()
omsh.DeepCopy(msh_d)
o_scalar = vtk.vtkFloatArray()
o_scalar.SetNumberOfComponents(1)

iod_scalar = msh_i_on_d.GetCellData().GetScalars()
d_scalar = msh_d.GetCellData().GetScalars()

for ix in range(msh_d.GetNumberOfCells()) : 
    value_assigned = 0

    if (d_scalar.GetTuple1(ix) == 0) : 
        value_assigned = -1

    else :  
        if (d_scalar.GetTuple1(ix) >= d_thres) : 
            value_assigned += 1

        if (iod_scalar.GetTuple1(ix) >= iod_thres) : 
            value_assigned += 2
    
    o_scalar.InsertNextTuple1(value_assigned)

omsh.GetCellData().SetScalars(o_scalar)
vtku.writeVtk(omsh, case_path, "overlap")


In [6]:
d_thres = 1.2
iod_thres = 1.15 # try different thresholds

omsh, Counts = vtku.fibrosisOverlapCell(msh_d, msh_i_on_d, d_thres, iod_thres, name0='dnav', name1='inav') 

fib_d = vtku.fibrorisScore(msh_d, d_thres)
fib_iod = vtku.fibrorisScore(msh_i_on_d, iod_thres)

tp = Counts['overlap']
tn = Counts['none']
fp = Counts['inav']
fn = Counts['dnav']

perf = iou.performanceMetrics(tp, tn, fp, fn)
print(perf)
print("D: {}, IOD: {}".format(fib_d, fib_iod))

{'jaccard': 0.4544549076421586, 'precision': 0.6279244338796447, 'recall': 0.6219330855018588, 'accuracy': 0.757652548167813}
D: 0.32589908001115137, IOD: 0.3209329907096264


# Project cell data
Full function definition (for debugging)

In [ ]:
import vtk 
import numpy as np

n = 1  # 0..22 (23 cases)
case_path = iou.fullfile(p2f, str(N[n]))
inav = iou.fullfile(case_path, xnav('i'), 'OUTPUT')
dnav = iou.fullfile(case_path, xnav('d'), 'OUTPUT')


dpath = iou.searchFileByType(dnav, 'Normalised', 'vtk')[0]
ipath = iou.searchFileByType(inav, 'Normalised', 'vtk')[0]

iou.cout("SOURCE: {}".format(dpath))
iou.cout("TARGET: {}".format(ipath))

msh_source = vtku.readVtk(dpath)  # source: dNAV
msh_target = vtku.readVtk(ipath)  # target: iNAV

omsh = vtk.vtkPolyData()
omsh.DeepCopy(msh_source)

target_pl = vtk.vtkCellLocator()
target_pl.SetDataSet(msh_target)
target_pl.AutomaticOn()
target_pl.BuildLocator()

target_scalar = msh_target.GetCellData().GetScalars()
o_scalar = vtk.vtkFloatArray()
o_scalar.SetNumberOfComponents(1)

default_value = 0
cog = vtku.getCentreOfGravity(msh_source)
for ix in range(msh_source.GetNumberOfCells()):
    pt = cog[ix, :]
    closest_pt = np.zeros((3, 1))
    c_id = np.int8()
    subid = np.int8()
    dist2 = np.float32()
    id_on_target = vtk.reference(c_id)
    Subid = vtk.reference(subid)
    Dist2 = vtk.reference(dist2)
    # target_pl.FindCell(pt)
    target_pl.FindClosestPoint(pt, closest_pt, id_on_target, Subid, Dist2)
    if (id_on_target > 0) : 
        mapped_val = target_scalar.GetTuple1(id_on_target) 
    else: 
        mapped_val = default_value

    o_scalar.InsertNextTuple1(mapped_val)

omsh.GetCellData().SetScalars(o_scalar)

vtku.writeVtk(omsh, case_path, "iod_test")